In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 45.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=24a907b8eb128bc589030eadc2e840a250dbbf556670952eae4ebb3a3819eca6
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('find_hacker').getOrCreate()

from pyspark.ml.clustering import KMeans

In [ ]:
data=spark.read.csv('/content/hack_data.csv',header=True,inferSchema=True)

In [ ]:
data.head()

Row(Session_Connection_Time=8, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7, Location='Slovenia', WPM_Typing_Speed=72.37)

In [ ]:
data.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                      1|              10.0|                 0|              1.0|                 6|Afghanistan|              40.0|
|    max|           

In [ ]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [ ]:
from pyspark.ml.linalg import Vectors

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
feat_cols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used',
'Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed']

In [ ]:
vector_assembler=VectorAssembler(inputCols=feat_cols,outputCol='features')

In [ ]:
final_data=vector_assembler.transform(data)

In [ ]:
from pyspark.ml.feature import StandardScaler


In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, )

In [ ]:
scalerModel = scaler.fit(final_data)

In [ ]:
cluster_final_data = scalerModel.transform(final_data)
kmeans4 = KMeans(featuresCol='scaledFeatures',k=4)
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)


In [ ]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)
model_k4 = kmeans4.fit(cluster_final_data)


In [ ]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



In [ ]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [ ]:
model_k4.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         3|   79|
|         2|   88|
|         0|   84|
+----------+-----+

